# Candidates of keyword

## Load model

In [1]:
import json
import os

from pl_module import get_pl_keywords
import requests

## Load data

In [8]:
def get_all_raw_pl():
     
    print("load data")
    total_data = [] 
    r = requests.get('https://steevebot.ml/all')
    ori_data = r.json()
    key = list(ori_data.keys())
    
    for k in key: 
        data = []
    
        for num, job_num in enumerate(ori_data[k]):
            if num%500 ==0:
                print(num)

            pl_des = get_pl_keywords(job_num["jobDescription"])
            pl_ski = get_pl_keywords(job_num["skills"])
            data.append(pl_des+pl_ski)

        total_data.append(data)
    return total_data

In [9]:
from DNN_data import get_Dnn_model
total_data = get_all_raw_pl()
# get_Dnn_model(total_data)

load data
0
500
1000
1500
2000
2500
3000
3500
0
500
1000


KeyboardInterrupt: 

In [ ]:
#### Return jobID, Field,PL[]